# Databases and SQL

A relational database is a collection of tables (and of relationships among them).  A table is simply a collection of rows, not unlike the matrices we've been working with.  However, a table also has associated with it a fixed schema consisting of column names and column types.  For example, imagine a users data set containg for each user her user is, name, and number of friends:

```python
users = [[0, "Hero", 0],
         [1, "Dunn", 2],
         [2, "Sue", 3],
         [3, "Chi", 3]]
```

In SQL, we might create this table with:

```sql
CREATE TABLE usrs (
    user_id INT NOT NULL,
    name VARCHAR(200),
    num_friends INT);
```

Notice that we specified that the `user_id` and `num_friends` must be integers and that the name should be a string of length 200 or less.  You can insert the rows with `INSERT` statements:

```sql
INSERT INTO users (user_id, name, num_friends) VALUES (0, 'Hero', 0);
```

Notice also that SQL statements need to end with semicolons, and that SQL requires single quotes for its strings.

In [2]:
import math, random, re
from collections import defaultdict

class Table:
    def __init__(self, columns):
        self.columns = columns
        self.rows = []

    def __repr__(self):
        """pretty representation of the table: columns then rows"""
        return str(self.columns) + "\n" + "\n".join(map(str, self.rows))

    def insert(self, row_values):
        if len(row_values) != len(self.columns):
            raise TypeError("wrong number of elements")
        row_dict = dict(zip(self.columns, row_values))
        self.rows.append(row_dict)

    def update(self, updates, predicate):
        for row in self.rows:
            if predicate(row):
                for column, new_value in updates.items():
                    row[column] = new_value

    def delete(self, predicate=lambda row: True):
        """delete all rows matching predicate
        or all rows if no predicate supplied"""
        self.rows = [row for row in self.rows if not(predicate(row))]

    def select(self, keep_columns=None, additional_columns=None):

        if keep_columns is None:         # if no columns specified,
            keep_columns = self.columns  # return all columns

        if additional_columns is None:
            additional_columns = {}

        # new table for results
        result_table = Table(keep_columns + list(additional_columns.keys()))

        for row in self.rows:
            new_row = [row[column] for column in keep_columns]
            for column_name, calculation in additional_columns.items():
                new_row.append(calculation(row))
            result_table.insert(new_row)

        return result_table

    def where(self, predicate=lambda row: True):
        """return only the rows that satisfy the supplied predicate"""
        where_table = Table(self.columns)
        where_table.rows = list(filter(predicate, self.rows))
        return where_table

    def limit(self, num_rows=None):
        """return only the first num_rows rows"""
        limit_table = Table(self.columns)
        limit_table.rows = (self.rows[:num_rows]
                            if num_rows is not None
                            else self.rows)
        return limit_table

    def group_by(self, group_by_columns, aggregates, having=None):

        grouped_rows = defaultdict(list)

        # populate groups
        for row in self.rows:
            key = tuple(row[column] for column in group_by_columns)
            grouped_rows[key].append(row)

        result_table = Table(group_by_columns + list(aggregates.keys()))

        for key, rows in grouped_rows.items():
            if having is None or having(rows):
                new_row = list(key)
                for aggregate_name, aggregate_fn in aggregates.items():
                    new_row.append(aggregate_fn(rows))
                result_table.insert(new_row)

        return result_table

    def order_by(self, order):
        new_table = self.select()       # make a copy
        new_table.rows.sort(key=order)
        return new_table

    def join(self, other_table, left_join=False):

        join_on_columns = [c for c in self.columns           # columns in
                           if c in other_table.columns]      # both tables

        additional_columns = [c for c in other_table.columns # columns only
                              if c not in join_on_columns]   # in right table

        # all columns from left table + additional_columns from right table
        join_table = Table(self.columns + additional_columns)

        for row in self.rows:
            def is_join(other_row):
                return all(other_row[c] == row[c] for c in join_on_columns)

            other_rows = other_table.where(is_join).rows

            # each other row that matches this one produces a result row
            for other_row in other_rows:
                join_table.insert([row[c] for c in self.columns] +
                                  [other_row[c] for c in additional_columns])

            # if no rows match and it's a left join, output with Nones
            if left_join and not other_rows:
                join_table.insert([row[c] for c in self.columns] +
                                  [None for c in additional_columns])

        return join_table
    
users = Table(["user_id", "name", "num_friends"])
users.insert([0, "Hero", 0])
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])

print("users table")
print(users)
print()

users table
['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}



Sometimes you need to update the data that's already in the database.  For instance, if Dunn acquired another friend, you might need to:

```sql
UPDATE users
SET num_friends = 3
WHERE user_id = 1;
```

The key features are:

- what table to update
- which rows to update
- which fields to update
- what their new values should be

Which will look like:

In [6]:
users.update({'num_friends': 3}, lambda row: row['user_id'] == 1)
print(users)

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 3}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}


There are two ways to delete rows from a table in SQL.  The dangerous way deletes every row from a table:

```sql
DELETE FROM users;
```

The less dangerous way adds a `WHERE` clause and only deletes rows that matcha certain condition:

```sql
DELETE FROM users WHERE user_id = 1;
```

Here's how it can work:

In [7]:
users.delete(lambda row: row["user_id"] == 1)
print(users)

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}


Typically, you don't inspect SQLn tables directly.  Instead you query them with a `SELECT` statement:

```sql
SELECT * from users;
SELECT * from users LIMIT 2;
SELECT user_id FROM users;
SELECT user_id FROM users WHERE name = 'Dunn';
```

You can also use `SELECT` statements to calculate fields:

```sql
SELECT LENGTH(name) AS name_length FROM users;
```

Here's how it will work:

In [8]:
print("users.select()")
print(users.select())
print()

print("users.limit(2)")
print(users.limit(2))
print()

print("users.select(keep_columns=[\"user_id\"])")
print(users.select(keep_columns=["user_id"]))
print()

print('where(lambda row: row["name"] == "Dunn")')
print(users.where(lambda row: row["name"] == "Dunn")
           .select(keep_columns=["user_id"]))
print()

def name_len(row): return len(row["name"])

print('with name_length:')
print(users.select(keep_columns=[],
                   additional_columns = { "name_length" : name_len }))
print()

users.select()
['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}

users.limit(2)
['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}

users.select(keep_columns=["user_id"])
['user_id']
{'user_id': 0}
{'user_id': 1}
{'user_id': 2}
{'user_id': 3}
{'user_id': 4}
{'user_id': 5}
{'user_id': 6}
{'user_id': 7}
{'user_id': 8}
{'user_id': 9}
{'user_id': 10}

where(lambda row: row["name"] == "Dunn")


Another common SQL operation is `GROUP BY`, which groups together rows with identical values in specified columns and produces aggregate values like `MIN` and `MAX` and `COUBT` and `SUM`.  For example, you might want to find the number of users and the smalled `user_id` for each possible name length:

```sql
SELECT LENGTH(name) as name_length,
    MIN(user_id) AS min_user_id,
    COUNT(*) AS num_users
FROM users
GROUP BY LENGTH(name);
```

Every field we `SELECT` needs to be either in the `GROUP BY` clause (which `name_length` is) or an aggregate computation (which `min_user_id` and `num_users` are).  SQL also supports a `HAVING` clause that behaves similarly to a `WHERE` clause except that its filter is applied to the aggregates.  You might want to know the average number of friends for users whose names start with specific letters but only see the results for letters whose corresponding average is greater than 1.

```sql
SELECT SUBSTR(name, 1, 1) AS first_letter,
    AVG(num_friends) AS avg_num_friends
FROM users
GROUP BY SUBSTR(name, 1, 1)
HAVING AVG(num_friends) > 1;
```

You can also compute overall aggregates.  In that case, you leave off the `GROUP BY`:

```sql
SELECT SUM(user_id) as user_id_sum
FROM users
WHERE user_id > 1
```

Which looks like:

In [9]:
def min_user_id(rows): return min(row["user_id"] for row in rows)

stats_by_length = users \
    .select(additional_columns={"name_len" : name_len}) \
    .group_by(group_by_columns=["name_len"],
              aggregates={ "min_user_id" : min_user_id,
                           "num_users" : len })

print("stats by length")
print(stats_by_length)
print()

def first_letter_of_name(row):
    return row["name"][0] if row["name"] else ""

def average_num_friends(rows):
    return sum(row["num_friends"] for row in rows) / len(rows)

def enough_friends(rows):
    return average_num_friends(rows) > 1

avg_friends_by_letter = users \
    .select(additional_columns={'first_letter' : first_letter_of_name}) \
    .group_by(group_by_columns=['first_letter'],
              aggregates={ "avg_num_friends" : average_num_friends },
              having=enough_friends)

print("avg friends by letter")
print(avg_friends_by_letter)
print()

def sum_user_ids(rows): return sum(row["user_id"] for row in rows)

user_id_sum = users \
    .where(lambda row: row["user_id"] > 1) \
    .group_by(group_by_columns=[],
              aggregates={ "user_id_sum" : sum_user_ids })

print("user id sum")
print(user_id_sum)
print()


stats by length
['name_len', 'min_user_id', 'num_users']
{'name_len': 4, 'min_user_id': 0, 'num_users': 4}
{'name_len': 3, 'min_user_id': 2, 'num_users': 3}
{'name_len': 5, 'min_user_id': 5, 'num_users': 4}

avg friends by letter
['first_letter', 'avg_num_friends']
{'first_letter': 'H', 'avg_num_friends': 1.5}
{'first_letter': 'D', 'avg_num_friends': 2.0}
{'first_letter': 'S', 'avg_num_friends': 3.0}
{'first_letter': 'C', 'avg_num_friends': 2.5}
{'first_letter': 'T', 'avg_num_friends': 3.0}
{'first_letter': 'K', 'avg_num_friends': 2.5}

user id sum
['user_id_sum']
{'user_id_sum': 54}



Frequently you'll want to sort your results.  For example, you might want to know the first two names of your users:

```sql
SELECT * FROM users
ORDER BY name
LIMIT 2;
```

This is easy to implement:

In [10]:
friendliest_letters = avg_friends_by_letter \
    .order_by(lambda row: -row["avg_num_friends"]) \
    .limit(4)

print("friendliest letters")
print(friendliest_letters)
print()

friendliest letters
['first_letter', 'avg_num_friends']
{'first_letter': 'S', 'avg_num_friends': 3.0}
{'first_letter': 'T', 'avg_num_friends': 3.0}
{'first_letter': 'C', 'avg_num_friends': 2.5}
{'first_letter': 'K', 'avg_num_friends': 2.5}



Relational database tables are often normalized, which means that they are organized to minimize redundancy.  For example, when we work with our users' interests in Python we can just give each user a `list` containing his interests.  SQL tables can't typically contain lists, so the typical solution is to create a second table `user_interests` containing the one-to-many relationship between `user_ids` and interests.  In SQL you might do:

```sql
CREATE TABLE user_interests (
    user_id INT NOT NULL,
    interest VARCHAR(100) NOT NULL
)
```

we'll do this for our purposes:

In [11]:
user_interests = Table(["user_id", "interest"])
user_interests.insert([0, "SQL"])
user_interests.insert([0, "NoSQL"])
user_interests.insert([2, "SQL"])
user_interests.insert([2, "MySQL"])

When our data lives across different tables, how to we analyze it?  By joining the tables together.  A `JOIN` combines rows in the left table with corresponding rows in the right table, where the meaning of corresponding is based on how we specify the join.  For example, to find the users interested in SQL, you'd query:

```sql
SELECT users.name
FROM users
JOIN user_interests
ON users.user_id = user_interests.user_id
WHERE user_interests.interest = 'SQL'
```

The `JOIN` says that, for each row in `users`, we should look at the `user_id` and associate that row with every row in `user_intrests` containing the same `user_id`.  Notice we had to specify which tables to `JOIN` and also which loumns to join `ON`.  This is an `INNER JOIN`, which returns the combinations of rows that match according to the speicified join criteria.  There is also a `LEFT JOIN`, which in addition to the combinations of matching rows, returns a row for each left-table row with no matching rows.  Using a `LEFT JOIN`, it's easy to count the number of interests each user has:

```sql
SELECT users.id, COUNT(user_interests.interest) AS num_interests
FROM users
LEFT JOIN user_interests
ON users.user_id = user_interests.user_id
```

The `LEFT JOIN` ensures that users with no interests will still have rows in the joined data set, and `COUNT` only counts vallues that are non-null.  Here's our implementation:

In [12]:
sql_users = users \
.join(user_interests) \
.where(lambda row: row["interest"] == "SQL") \
.select(keep_columns=["name"])

print("sql users")
print(sql_users)
print()

def count_interests(rows):
    """counts how many rows have non-None interests"""
    return len([row for row in rows if row["interest"] is not None])

user_interest_counts = users \
    .join(user_interests, left_join=True) \
    .group_by(group_by_columns=["user_id"],
              aggregates={"num_interests" : count_interests })

print("user interest counts")
print(user_interest_counts)

sql users
['name']
{'name': 'Hero'}
{'name': 'Sue'}

user interest counts
['user_id', 'num_interests']
{'user_id': 0, 'num_interests': 2}
{'user_id': 1, 'num_interests': 0}
{'user_id': 2, 'num_interests': 2}
{'user_id': 3, 'num_interests': 0}
{'user_id': 4, 'num_interests': 0}
{'user_id': 5, 'num_interests': 0}
{'user_id': 6, 'num_interests': 0}
{'user_id': 7, 'num_interests': 0}
{'user_id': 8, 'num_interests': 0}
{'user_id': 9, 'num_interests': 0}
{'user_id': 10, 'num_interests': 0}


In SQL, you can `SELECT` from and `JOIN` the results of queries as if they were tables.  So if you wanted to find the smallest `user_id` of anyone interested in SQL, you could use a subquery.

```sql
SELECT MIN(user_id) AS min_user_id FROM
(SELECT user_id FROM user_interests WHERE interest = 'SQL') sql_interests;
```

Here's what it looks like in our implementation:

In [13]:
likes_sql_user_ids = user_interests \
    .where(lambda row: row["interest"] == "SQL") \
    .select(keep_columns=['user_id'])

likes_sql_user_ids.group_by(group_by_columns=[],
                            aggregates={ "min_user_id" : min_user_id })

print("likes sql user ids")
print(likes_sql_user_ids)

likes sql user ids
['user_id']
{'user_id': 0}
{'user_id': 2}
